In [ ]:
import pandas as pd

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f transport-all-comments.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/transport-all-comments.csv.xz")


In [ ]:
posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

In [ ]:
posts.dtypes

In [ ]:
len(posts)

In [ ]:
top = posts[posts["parent_id"].isna()]
len(top)

In [ ]:
posts["author"].nunique()

In [ ]:
top["author"].nunique()

In [ ]:
top.groupby("author").count()["id"].sort_values(ascending=False).head(20)

In [ ]:
posts.groupby("author").count()["id"].sort_values(ascending=False).head(20)

In [ ]:
posts.set_index("created_utc").resample("M").count()["id"].plot(title="Gesamtposts pro Monat")

In [ ]:
posts.set_index("created_utc").resample("Q").count()["id"].plot(title="Gesamtposts pro Quartal")

In [ ]:
top.set_index("created_utc").resample('Q').count()["id"].plot(title="Toplevel-Posts pro Quartal")

In [ ]:
time_author = posts.set_index("created_utc").resample("Q").agg({"author": pd.Series.nunique })

In [ ]:
time_author.plot(title="Autoren pro Quartal")

In [ ]:
posts.set_index("created_utc").resample("M").count()["id"].plot(figsize=(16, 9), title="Total posts per month")

In [ ]:
cpa = posts.groupby("author").count()[["id"]]

In [ ]:
cpa = cpa[~cpa.index.isin(["[deleted]", "AutoModerator"])]

In [ ]:
cpa.describe()

In [ ]:
cpa.plot.hist(bins=80, logy=True)

In [ ]:
cc = posts[["id", "parent_id"]].groupby("parent_id").count().rename(columns={"id": "comment_count"})

In [ ]:
comments_and_score = top.merge(cc, left_on="id", right_index=True, how="left").\
                         fillna({"comment_count": 0})
comments_and_score.plot.scatter(x="score", y="comment_count")

In [ ]:
import scipy.stats
r = scipy.stats.linregress(comments_and_score["score"], comments_and_score["comment_count"])
r